## Example on how to evaluate a vision encoder with the Hummingbird or Dense NN Retrieval Evaluation

<a href="https://githubtocolab.com/vpariza/open-hummingbird-eval/blob/main/examples/hbird_eval_example_faiss_gpu.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### 1. Install required Libraries

In [1]:
!pip uninstall -y thinc # needed when running on Google Colab, avoids a compatibility issue with numpy

!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121
!pip install lightning==2.4.0
!pip install torchmetrics==1.7.0
!pip install tqdm==4.67.1 # optional
!pip install scipy==1.15.2
!pip install joblib==1.4.2
!pip install numpy==1.26.4
!pip install triton==2.2.0
!pip install faiss-gpu-cu12

!pip install scann==1.3.5 # this is needed when using nn_method='scann'

# ToDo: Restart the session after running this cell and rerun the cell to
# Make sure you don't see the following error after restarting the session and rerunning the rell:
# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
# spacy 3.8.5 requires thinc<8.4.0,>=8.3.4, which is not installed.

Looking in indexes: https://download.pytorch.org/whl/cu121


### 2. Clone our repository

In [2]:
!git clone https://github.com/vpariza/open-hummingbird-eval.git

Cloning into 'open-hummingbird-eval'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 194 (delta 37), reused 82 (delta 27), pack-reused 87 (from 1)
Receiving objects: 100% (194/194), 709.36 KiB | 2.18 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
# Move to the repository folder
%cd open-hummingbird-eval

/content/open-hummingbird-eval


### 3. Install repo

In [3]:
# ToDo: Install this after no error in the dependency installation and restarting the session.
# !pip install .  # originally this was used but the evaluation errors as "ModuleNotFoundError: No module named 'hbird'"
!pip install -e .  # install in editable mode so that subprocesses find it


Obtaining file:///content/open-hummingbird-eval
  Preparing metadata (setup.py) ... done
  Attempting uninstall: open-hummingbird-eval
    Found existing installation: open-hummingbird-eval 1.0.0
    Uninstalling open-hummingbird-eval-1.0.0:
      Successfully uninstalled open-hummingbird-eval-1.0.0
  Running setup.py develop for open-hummingbird-eval


This is the error log if `!pip install .` is used instead of `!pip install -e .`:
```bash
---------------------------------------------------------------------------
_RemoteTraceback                          Traceback (most recent call last)
_RemoteTraceback:
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py", line 426, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'hbird'
"""

The above exception was the direct cause of the following exception:

BrokenProcessPool                         Traceback (most recent call last)
<ipython-input-8-762c7570422c> in <cell line: 0>()
----> 1 hbird_miou = hbird_evaluation(model.to(device),
      2         d_model=embed_dim,        # size of the embedding feature vectors of patches
      3         patch_size=patch_size,
      4         batch_size = batch_size,
      5         input_size=input_size,

11 frames
/usr/local/lib/python3.11/dist-packages/joblib/parallel.py in _return_or_raise(self)
    761         try:
    762             if self.status == TASK_ERROR:
--> 763                 raise self._result
    764             return self._result
    765         finally:

# BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
```

### 4. Download your preferred dataset

In [ ]:
# Used to delete previously downloaded and used
# !rm -rf ./VOCSegmentation
# !rm voc_data.zip

#### Option 1: Tiny Pascal VOC

In [ ]:
# Download the tiny Pascal VOC -
# https://1drv.ms/u/c/67fac29a77adbae6/Ed4Bdc53JIBBl1fP-39XylABFvmYKHKiEFnH-OVa8C_ZRg?e=R1VPhl # tiny old
# https://1drv.ms/u/c/67fac29a77adbae6/EbGBdN6Z9LNEt3-3FveU344BnlECl_cwueg8-getyattqA?e=HPrVa1 # tiny new (fixed)
# !wget -O voc_data.zip "<Paste the curl lnk of the onedrive object here>"
# ---

# You can directly get it from this link in google drive https://drive.google.com/file/d/1Pz5UhrkjXOKHesVsc7MRbaAtp6LP3hu_/view?usp=sharing
# using the commands:
# !pip install -q gdown
# !gdown --id 1Pz5UhrkjXOKHesVsc7MRbaAtp6LP3hu_ -O voc_data.zip

# NOTE: there is an issue with this one! The masks and images do not correspond!


#### Option 2: Mini Pascal VOC

In [ ]:
# Download the mini Pascal VOC -
# https://1drv.ms/u/c/67fac29a77adbae6/EUOf5pG-1phGkn6jFyzMnE8BD3NMPw1f8dqXNg9Qm6DIhA?e=PxS2we # mini old
# https://1drv.ms/u/c/67fac29a77adbae6/EXkWjXPBLmNIgqI1G8yZzBYB_11wyXI-_8u0pyERgib8fA?e=qle36E # mini new (fixed)
# !wget -O voc_data.zip "<Paste the curl lnk of the onedrive object here>"
# ---

# You can directly get it from this link in google drive https://drive.google.com/file/d/1-mOJbAsWPXc6DtGbzEUWLq5J55CzQDM1/view?usp=sharing
# using the commands:
# !pip install -q gdown
# !gdown --id 1-mOJbAsWPXc6DtGbzEUWLq5J55CzQDM1 -O voc_data.zip

#### Option 3: Pascal VOC

In [17]:
# Download the full Pascal VOC - https://1drv.ms/u/c/67fac29a77adbae6/EcilwP2YfSBGj3xvUWCu42EBfoHTmSlHz8Aw_8VgqwWd8g?e=KUFcva # full
# !wget -O voc_data.zip "<Paste the curl lnk of the onedrive object here>"
# ---

# You can directly get it from this link in google drive https://drive.google.com/file/d/1SxlQUjlIrPZpPXB0fTyOgjuXZgPwfvr6/view?usp=sharing
# using the commands:
!pip install -q gdown
!gdown --id 1SxlQUjlIrPZpPXB0fTyOgjuXZgPwfvr6 -O voc_data.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1SxlQUjlIrPZpPXB0fTyOgjuXZgPwfvr6
From (redirected): https://drive.google.com/uc?id=1SxlQUjlIrPZpPXB0fTyOgjuXZgPwfvr6&confirm=t&uuid=e619bb19-ed5b-4d9e-b74f-e4ef0ffa0121
To: /content/open-hummingbird-eval/voc_data.zip
100% 1.96G/1.96G [00:32<00:00, 60.0MB/s]


### 5. Unzip Contents of zip Dataset

In [18]:
# !rm -r ./datasets/voc_data  # remove folder if already existing
!mkdir -p ./datasets/voc_data
!unzip voc_data.zip -d ./datasets/voc_data

Streaming output truncated to the last 5000 lines.
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2011_000837.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2010_005167.png  
 extracting: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2008_006526.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2009_004133.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2011_000607.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2008_006576.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2010_005305.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2010_005991.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2010_004543.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug/2010_004417.png  
  inflating: ./datasets/voc_data/VOCSegmentation/SegmentationClassAug

### 6. Evaluate a preferred model on the downloaded dataset

In [3]:
# ToDo: If this fails restart the session and restart the session and rerun "3. Install repo"
import torch
from hbird.hbird_eval import hbird_evaluation

In [4]:
# Parameters for the model dino
device = 'cuda'
input_size = 224
# batch_size = 64  # 64 is too much for Google Colab, session crashes with "Out of RAM" message
batch_size = 16
patch_size = 16
embed_dim = 384
model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [5]:
# Dataset Configurations
dataset_name='voc'
data_dir='./datasets/voc_data/VOCSegmentation'
train_fs_path= './datasets/voc_data/VOCSegmentation/sets/trainaug.txt'
val_fs_path= './datasets/voc_data/VOCSegmentation/sets/val.txt'


In [6]:
fn = lambda model, imgs: (model.get_intermediate_layers(imgs)[0][:, 1:], None)

In [9]:
# # Check for missing images in case of issues:

# from pathlib import Path

# # Paths to image files
# img_dir = Path("./datasets/voc_data/VOCSegmentation/images")

# # Paths to the train and validation file sets
# val_fs_path = Path("./datasets/voc_data/VOCSegmentation/sets/val.txt")
# train_fs_path = Path("./datasets/voc_data/VOCSegmentation/sets/trainaug.txt")

# # Read image IDs from val.txt and trainaug.txt
# with open(val_fs_path) as f:
#     val_ids = [line.strip() for line in f]

# with open(train_fs_path) as f:
#     train_ids = [line.strip() for line in f]

# # List of image IDs in the images directory
# img_ids = set(p.stem for p in img_dir.glob("*.jpg"))

# # Total number of images in the images directory
# total_images = len(img_ids)

# # Number of images listed in val.txt
# num_val_images = len(val_ids)

# # Number of images in val.txt that are not found in the images directory
# missing_val_images = [id for id in val_ids if id not in img_ids]
# num_missing_val_images = len(missing_val_images)

# # Number of images listed in trainaug.txt
# num_train_images = len(train_ids)

# # Number of images in trainaug.txt that are not found in the images directory
# missing_train_images = [id for id in train_ids if id not in img_ids]
# num_missing_train_images = len(missing_train_images)

# # Output results
# print(f"🖼️ Total images in images folder: {total_images}")
# print(f"📄 Number of images listed in val.txt: {num_val_images}")
# print(f"❌ Number of images in val.txt not found in images: {num_missing_val_images}")
# print(f"📄 Number of images listed in trainaug.txt: {num_train_images}")
# print(f"❌ Number of images in trainaug.txt not found in images: {num_missing_train_images}")

# # Print some examples of missing images
# if num_missing_val_images > 0:
#     print(f"Missing images from val.txt: {missing_val_images[:5]}")
# if num_missing_train_images > 0:
#     print(f"Missing images from trainaug.txt: {missing_train_images[:5]}")


🖼️ Total images in images folder: 17125
📄 Number of images listed in val.txt: 1449
❌ Number of images in val.txt not found in images: 0
📄 Number of images listed in trainaug.txt: 10582
❌ Number of images in trainaug.txt not found in images: 0


In [ ]:
hbird_miou = hbird_evaluation(model.to(device),
        d_model=embed_dim,        # size of the embedding feature vectors of patches
        patch_size=patch_size,
        batch_size = batch_size,
        input_size=input_size,
        augmentation_epoch=1,     # how many iterations of augmentations to use on top of the training dataset in order to generate the memory
        device=device,
        return_knn_details=False, # whether to return additional NNs details
        nn_method='faiss',
        n_neighbours=30,         # the number of neighbors to fetch per image patch
        nn_params=None,           # Other parameters to be used for the k-NN operator
        ftr_extr_fn=fn,           # function that extracts features from a vision encoder on images
        dataset_name=dataset_name,       # the name of the dataset to use, currently only Pascal VOC is included.
        data_dir=data_dir,    # path to the dataset to use for evaluation
        memory_size=None,
        train_fs_path=train_fs_path,
        val_fs_path=val_fs_path,
        num_workers=2,  # Google Colab says it can only support 2 workers. This fixes a warning and a session crash. The default is 8.
        )

Found 10582 images and 10582 masks in ./datasets/voc_data/VOCSegmentation
Found 1449 images and 1449 masks in ./datasets/voc_data/VOCSegmentation
Train size 10582
Val size 1449


Augmentation loop: 100%|██████████| 1/1 [02:58<00:00, 178.15s/it]


Using Replicated index


Evaluation loop: 100%|██████████| 90/90 [03:11<00:00,  2.13s/it]


seg map preds have size (68278472,)
Parallelizing iou computation
took 244.04009366035461 seconds


In [ ]:
print('Hummingbird Evaluation (mIoU):', hbird_miou)


Hummingbird Evaluation (mIoU): 0.45851727419622407


In [7]:
hbird_miou = hbird_evaluation(model.to(device),
        d_model=embed_dim,        # size of the embedding feature vectors of patches
        patch_size=patch_size,
        batch_size = batch_size,
        input_size=input_size,
        augmentation_epoch=1,     # how many iterations of augmentations to use on top of the training dataset in order to generate the memory
        device=device,
        return_knn_details=False, # whether to return additional NNs details
        # nn_method='faiss',
        nn_method='scann',
        n_neighbours=30,         # the number of neighbors to fetch per image patch
        nn_params=None,           # Other parameters to be used for the k-NN operator
        ftr_extr_fn=fn,           # function that extracts features from a vision encoder on images
        dataset_name=dataset_name,       # the name of the dataset to use, currently only Pascal VOC is included.
        data_dir=data_dir,    # path to the dataset to use for evaluation
        memory_size=None,
        train_fs_path=train_fs_path,
        val_fs_path=val_fs_path,
        num_workers=2,  # Google Colab says it can only support 2 workers. This fixes a warning and a session crash. The default is 8.
        )

Found 10582 images and 10582 masks in ./datasets/voc_data/VOCSegmentation
Found 1449 images and 1449 masks in ./datasets/voc_data/VOCSegmentation
Train size 10582
Val size 1449


Evaluation loop:   0%|          | 0/90 [00:01<?, ?it/s]


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
print('Hummingbird Evaluation (mIoU):', hbird_miou)
